In [1]:
# This script is based on:
# https://www.tensorflow.org/get_started/mnist/pros

import sys
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import graph_util as gu
from tensorflow.python.framework.graph_util import remove_training_nodes
from tensorflow.tools.graph_transforms import TransformGraph

# Import training data

In [2]:
mnist = input_data.read_data_sets("mnist_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# Define Tensorflow Model

### Fully connected 2 layer NN

In [3]:
def deepnn(x):
    W_fc1 = weight_variable([784, 128], name='W_fc1')
    b_fc1 = bias_variable([128], name='b_fc1')
    a_fc1 = tf.add(tf.matmul(x, W_fc1), b_fc1, name="zscore")
    h_fc1 = tf.nn.relu(a_fc1)
    
    W_fc2 = weight_variable([128, 64], name='W_fc2')
    b_fc2 = bias_variable([64], name='b_fc2')
    a_fc2 = tf.add(tf.matmul(h_fc1, W_fc2), b_fc2, name="zscore")
    h_fc2 = tf.nn.relu(a_fc2)
    
    W_fc3 = weight_variable([64, 10], name='W_fc3')
    b_fc3 = bias_variable([10], name='b_fc3')
    logits = tf.add(tf.matmul(h_fc2, W_fc3), b_fc3, name="logits")
    y_pred = tf.argmax(logits, 1, name='y_pred')

    return y_pred, logits


def weight_variable(shape, name):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name)


def bias_variable(shape, name):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name)

### Specify inputs, outputs, and a cost function

In [4]:
# Reset default graph
tf.reset_default_graph()

# Create the model
x = tf.placeholder(tf.float32, [None, 784], name="x")

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10], name="y")

# Build the graph for the deep net
y_pred, logits = deepnn(x)

with tf.name_scope("Loss"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, 
                                                               logits=logits)
    loss = tf.reduce_mean(cross_entropy, name="cross_entropy_loss")
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss, name="train_step")
  
# Here we specify the output as "Prediction/y_pred", this will be important later
with tf.name_scope("Prediction"): 
    correct_prediction = tf.equal(y_pred, 
                                  tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")

# Train the model

In [5]:
batch_size = 100
n_epochs = 100
n_batches = int(mnist.train.num_examples / batch_size)
print(n_batches)

550


In [6]:
sess = tf.Session()
# Initialize the variables (i.e. assign their default value)
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

train_loss, train_accuracy = sess.run([loss, accuracy], feed_dict={x: mnist.train.images, y_: mnist.train.labels})
print('Epoch %d, training loss: %g, training accuracy: %g' % (0, train_loss, train_accuracy))
val_loss, val_accuracy = sess.run([loss, accuracy], feed_dict={x: mnist.validation.images,
                                                               y_: mnist.validation.labels})
print('Epoch %d, validation loss: %g, validation accuracy %g' % (0, val_loss, val_accuracy))

Epoch 0, training loss: 2.34705, training accuracy: 0.126764
Epoch 0, validation loss: 2.33597, validation accuracy 0.136


In [7]:
for i in range(n_epochs):
    for j in range(n_batches):
        batch_images, batch_labels = mnist.train.next_batch(batch_size)
        sess.run(train_step, feed_dict={x: batch_images, y_: batch_labels})
        
    if i % 10 == 0:
        train_loss, train_accuracy = sess.run([loss, accuracy], feed_dict={x: mnist.train.images, 
                                                                           y_: mnist.train.labels})
        print('Epoch %d, training loss: %g, training accuracy: %g' % (i, train_loss, train_accuracy))
        val_loss, val_accuracy = sess.run([loss, accuracy], feed_dict={x: mnist.validation.images,
                                                                       y_: mnist.validation.labels})
        print('Epoch %d, validation loss: %g, validation accuracy %g' % (i, val_loss, val_accuracy))

Epoch 0, training loss: 0.489011, training accuracy: 0.873782
Epoch 0, validation loss: 0.468535, validation accuracy 0.8812
Epoch 10, training loss: 0.138119, training accuracy: 0.960509
Epoch 10, validation loss: 0.143267, validation accuracy 0.9612
Epoch 20, training loss: 0.0820812, training accuracy: 0.976909
Epoch 20, validation loss: 0.10327, validation accuracy 0.9694
Epoch 30, training loss: 0.0511257, training accuracy: 0.986291
Epoch 30, validation loss: 0.08577, validation accuracy 0.9744
Epoch 40, training loss: 0.0327077, training accuracy: 0.991982
Epoch 40, validation loss: 0.0788695, validation accuracy 0.975
Epoch 50, training loss: 0.0203411, training accuracy: 0.995982
Epoch 50, validation loss: 0.075657, validation accuracy 0.9772
Epoch 60, training loss: 0.0123359, training accuracy: 0.998291
Epoch 60, validation loss: 0.0746826, validation accuracy 0.9778
Epoch 70, training loss: 0.00686135, training accuracy: 0.9994
Epoch 70, validation loss: 0.0797281, validati

## What is the final accuracy

In [8]:
print('test accuracy %g' % sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

test accuracy 0.9763


# Freeze the graph

In [9]:
saver.save(sess, "./chkps/mnist_model")
out_nodes = [y_pred.op.name]
print(out_nodes)

Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'W_fc1' has type str, but expected one of: int, long, bool
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'W_fc1' has type str, but expected one of: int, long, bool
['y_pred']


### Remove unnecessary training nodes

In [10]:
sub_graph_def = remove_training_nodes(sess.graph_def)

### Freeze Constants

In [11]:
sub_graph_def = gu.convert_variables_to_constants(sess, sub_graph_def, out_nodes)

INFO:tensorflow:Froze 6 variables.
INFO:tensorflow:Converted 6 variables to const ops.


### Save the graph to PB file

In [12]:
graph_path = tf.train.write_graph(sub_graph_def,
                                  "./mnist_model_0to9",
                                  "deep_mlp.pb",
                                  as_text=False)

print('written graph to: %s' % graph_path)

written graph to: ./mnist_model_0to9/deep_mlp.pb


In [13]:
sess.run(y_pred, feed_dict={x: mnist.test.images[0:100, :]})

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4,
       6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9])

In [14]:
sess.run(y_pred, feed_dict={x: mnist.test.images[[3, 2, 1, 18, 4, 15, 11, 0, 61, 7], :]})

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [15]:
mnist.test.labels[[3, 2, 1, 18, 4, 15, 11, 0, 61, 7], :]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [16]:
# close session
sess.close()